In [21]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F

In [35]:
@torch.no_grad()
def match_template_torch(img, template):
    img_tensor = torch.from_numpy(img).unsqueeze(dim=0).unsqueeze(dim=0).type(torch.float32)
    template_tensor = torch.from_numpy(template).unsqueeze(dim=0).unsqueeze(dim=0).type(torch.float32)
    result = match_template_torch_core(img_tensor, template_tensor)
    return result.cpu().numpy()

@torch.jit.script
def match_template_torch_core(img_tensor, template_tensor):
    result1 = torch.nn.functional.conv2d(img_tensor, template_tensor, bias=None, stride=1, padding=0)
    result2 = torch.sqrt(torch.sum(template_tensor**2)* torch.nn.functional.conv2d(img_tensor**2, torch.ones_like(template_tensor), bias=None, stride=1, padding=0))

    return (result1/result2).squeeze(0).squeeze(0)

In [36]:
img_ndarray = np.arange(16).reshape(4, 4).astype(np.uint8)
temp_ndarray = np.arange(4).reshape(2, 2).astype(np.uint8)
img_ndarray_norm = np.linalg.norm(img_ndarray, ord=2)
temp_ndarray_norm = np.linalg.norm(temp_ndarray, ord=2)

In [40]:
result_ndarray = cv2.matchTemplate(img_ndarray, temp_ndarray, cv2.TM_CCORR_NORMED)
result_ndarray

array([[0.98974335, 0.9869275 , 0.9719086 ],
       [0.94063413, 0.927731  , 0.9166985 ],
       [0.8991574 , 0.8921426 , 0.8860286 ]], dtype=float32)

In [41]:
result_tensor = match_template_torch(img=img_ndarray, template=temp_ndarray)
result_tensor

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


array([[0.98974335, 0.98692757, 0.9719086 ],
       [0.94063413, 0.927731  , 0.9166985 ],
       [0.8991574 , 0.8921426 , 0.88602865]], dtype=float32)